<h1 style="text-align:center;color:#0F4C81">Multiple Linear Regression</h1>

Consider this, suppose you have to estimate the price of a certain house you want to buy. You know the floor area, the age of the house, its distance from your workplace, the crime rate of the place, etc.

Now, some of these factors will affect the price of the house positively. For example more the area, the more the price. On the other hand, factors like distance from the workplace, and the crime rate can influence your estimate of the house negatively.

*Disadvantages of Simple Linear Regression*: Running separate simple linear regressions will lead to different outcomes when we are interested in just one. Besides that, there may be an input variable that is itself correlated with or dependent on some other predictor. This can cause wrong predictions and unsatisfactory results.

This is where Multiple Linear Regression comes into the picture.

$$
Y = \beta_0 + \beta_1X_1 + \beta_2X_2 + \dots + \beta_mX_m + \varepsilon
$$

Here, Y is the output variable, and X terms are the corresponding input variables. Notice that this equation is just an extension of *Simple Linear Regression*, and each predictor has a corresponding slope coefficient ($\beta$).

The first $\beta$ term ($\beta_0$) is the intercept constant and is the value of $Y$ in absence of all predictors (i.e when all $X$ terms are 0). It may or may or may not hold any significance in a given regression problem. It’s generally there to give a relevant nudge to the line/plane of regression.

Let’s now understand this with the help of some data.

### Visualizing the data

We are going to use `Advertising` data which is available on the site of USC Marshall School of Business.

The advertising data set consists of the sales of a product in 200 different markets, along with advertising budgets for three different media: TV, radio, and newspaper. Here’s how it looks like:

In [10]:
import pandas as pd; df=pd.read_csv('data/advertising.csv'); display(df.head()); df.shape

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9


(200, 4)

The first row of the data says that the advertising budgets for TV, radio, and newspaper were $230.1k, $37.8k, and $69.2k respectively, and the corresponding number of units that were sold was 22.1k (or 22,100).

In Simple Linear Regression, we can see how each advertising medium affects sales when applied without the other two media. However, in practice, all three might be working together to impact net sales. We did not consider the combined effect of these media on sales.

Multiple Linear Regression solves the problem by taking account of all the variables in a single expression. Hence, our Linear Regression model can now be expressed as:

$$
\text{sales} = \beta_0 + \beta_1\text{TV} + \beta_2\text{radio} + \beta_3\text{newspaper}
$$

Finding the values of these constants ($\beta$) is what regression model does by minimizing the error function and fitting the best line or *hyperplane* (depending on the number of input variables).

This is done by minimizing the Residual Sum of Squares (RSS), which is obtained by squaring the differences between actual and predicted outcomes.

$$
\text{RSS} = \sum_i^n \left(y_i - \hat{y_i} \right)^2
$$

Because this method finds the least sum of squares, it is also known as the **Ordinary Least Squares** (OLS) method.

Here is how we can find coefficients of multiple linear regression using `sklearn` library:

In [24]:
from sklearn.linear_model import LinearRegression

cols = ['TV', 'Radio', 'Newspaper']
X = df[cols]
y = df['Sales']
model = LinearRegression()
model.fit(X, y)

print('Intercept:', model.intercept_)

for name, coef in zip(cols, model.coef_):
    print(f'{name:<10}: {coef}')

Intercept: 4.625124078808653
TV        : 0.05444578033757095
Radio     : 0.1070012282387029
Newspaper : 0.0003356579223305718


Now that we have these values, how to interpret them? Here’s how:

- If we fix the budget for TV & newspaper, then increasing the radio budget by $1000 will lead to an increase in sales by around 107 units ($0.107 \cdot 1000$).

- Similarly, by fixing the radio & newspaper, we infer an approximate rise of 54 units of products per $1000 increase in the TV budget.

- However, for the newspaper budget, since the coefficient is quite negligible (close to zero), it’s evident that the newspaper is not affecting the sales.

**Evaluating the Model**

In [38]:
from sklearn import metrics

y_pred = model.predict(X)
meanAbErr = metrics.mean_absolute_error(y, y_pred)
meanSqErr = metrics.mean_squared_error(y, y_pred)
rootMeanSqErr = metrics.root_mean_squared_error(y, y_pred)
print('R squared: {:.2f}'.format(model.score(X, y)*100))
print('Mean Absolute Error:', meanAbErr)
print('Mean Square Error:', meanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)

R squared: 90.26
Mean Absolute Error: 1.2363919943957848
Mean Square Error: 2.706006147627316
Root Mean Square Error: 1.6449942697855564


<h2 style="text-align:center;color:#0F4C81">Solving multiple linear regression with pure Python</h2>

$$
\beta = (X^TX)^{-1}X^Ty
$$

Dataset: `data/advertising.csv`

In [47]:
# solving multiple linear regression with pure python
import csv

data_file = 'data/advertising.csv'

with open(data_file) as f:
    reader = csv.reader(f)
    columns = next(reader)

    data = [[float(i) for i in row] for row in reader]

X = [row[:-1] for row in data]
y = [row[-1] for row in data]

# Add a column of ones to X for the intercept term
for row in X:
    row.insert(0, 1.0)

# Transpose X for matrix calculations
def transpose(matrix):
    return list(map(list, zip(*matrix)))

# Matrix multiplication
def matmul(A, B):
    result = [
        [sum(a * b for a, b in zip(A_row, B_col)) for B_col in zip(*B)]
        for A_row in A
    ]
    return result

# Invert a square matrix
def invert(matrix):
    n = len(matrix)
    identity = [[1 if i == j else 0 for j in range(n)] for i in range(n)]

    for i in range(n):
        diag_element = matrix[i][i]
        for j in range(n):
            matrix[i][j] /= diag_element
            identity[i][j] /= diag_element
        for k in range(n):
            if k != i:
                factor = matrix[k][i]
                for j in range(n):
                    matrix[k][j] -= factor * matrix[i][j]
                    identity[k][j] -= factor * identity[i][j]
    return identity

X_t = transpose(X)
X_t_X = matmul(X_t, X)
X_t_y = matmul(X_t, [[yi] for yi in y])
X_t_X_inv = invert(X_t_X)
beta = matmul(X_t_X_inv, X_t_y)

coef = [b[0] for b in beta]

print(f"Intercept: {coef[0]}")
print(f"TV: {coef[1]}")
print(f"Radio: {coef[2]}")
print(f"Newspaper: {coef[3]}")

Intercept: 4.6251240788087
TV: 0.05444578033757067
Radio: 0.10700122823870217
Newspaper: 0.00033565792233059


<h2 style="text-align:center;color:#0F4C81">Solving multiple linear regression with NumPy</h2>

In [64]:
import numpy as np

data_file = 'data/advertising.csv'

data = np.loadtxt(data_file, skiprows=1, delimiter=',')

X = data[:,:-1]
y = data[:, -1]

X = np.c_[np.ones(X.shape[0]), X]

coef = np.linalg.inv(X.T @ X) @ X.T @ y

print(f"Intercept: {coef[0]}")
print(f"TV: {coef[1]}")
print(f"Radio: {coef[2]}")
print(f"Newspaper: {coef[3]}")

Intercept: 4.625124078808676
TV: 0.05444578033757087
Radio: 0.10700122823870181
Newspaper: 0.00033565792233131164
